In [68]:
#!pip install trafilatura, bs4
"""
On my mac at least i needed to
#pip install -U charset_normalizer
#pip uninstall charset_normalizer -y
#!pip install lxml_html_clean
"""
import requests
import trafilatura
from bs4 import BeautifulSoup

def fetch_text(uri):
    return trafilatura.extract(trafilatura.fetch_url(uri))

"""the simplest possible text scrape"""
site = 'https://docs.fastht.ml/'
downloaded = trafilatura.fetch_url(site)
home_content = trafilatura.extract(downloaded)
soup = BeautifulSoup(downloaded, 'html.parser')
links = soup.find_all('a')
all_links = set([link.get('href') for link in links if link.get('href','-')[0]=='.'])
all_links = [f"{site}{l[2:]}" for l in all_links]
all_pages = {uri: fetch_text(uri) for uri in all_links }
for k,v in all_pages.items():
    print(k, len(v))
home_content

https://docs.fastht.ml/explains/oauth.html 9225
https://docs.fastht.ml/api/oauth.html 902
https://docs.fastht.ml/explains/faq.html 3307
https://docs.fastht.ml/ref/live_reload.html 1206
https://docs.fastht.ml/api/components.html 4443
https://docs.fastht.ml/explains/routes.html 5174
https://docs.fastht.ml/tutorials/index.html 519
https://docs.fastht.ml/api/js.html 3647
https://docs.fastht.ml/explains/explaining_xt_components.html 5224
https://docs.fastht.ml/api/core.html 17507
https://docs.fastht.ml/api/fastapp.html 3506
https://docs.fastht.ml/tutorials/by_example.html 46063
https://docs.fastht.ml/tutorials/quickstart_for_web_devs.html 14063
https://docs.fastht.ml/api/pico.html 4002
https://docs.fastht.ml/index.html 3249
https://docs.fastht.ml/tutorials/tutorial_for_web_devs.html 6624
https://docs.fastht.ml/ref/defining_xt_component.html 4456
https://docs.fastht.ml/tutorials/e2e.html 12060
https://docs.fastht.ml/api/cli.html 377
https://docs.fastht.ml/api/xtend.html 5530


'FastHTML\nWelcome to the official FastHTML documentation.\nFastHTML is a new next-generation web framework for fast, scalable web applications with minimal, compact code. It’s designed to be:\n- Powerful and expressive enough to build the most advanced, interactive web apps you can imagine.\n- Fast and lightweight, so you can write less code and get more done.\n- Easy to learn and use, with a simple, intuitive syntax that makes it easy to build complex apps quickly.\nFastHTML apps are just Python code, so you can use FastHTML with the full power of the Python language and ecosystem. FastHTML’s functionality maps 1:1 directly to HTML and HTTP, but allows them to be encapsulated using good software engineering practices—so you’ll need to understand these foundations to use this library fully. To understand how and why this works, please read this first: about.fastht.ml.\nInstallation\nSince fasthtml\nis a Python library, you can install it with:\npip install python-fasthtml\nIn the near

In [78]:
"""do some lame chunking"""
def chunk(s, l=10000):
    for i in range(0, len(s), l):
        yield i, s[i:i + l] 
        
chunks = []
for uri, content in all_pages.items():
    for i, c in chunk(content):
        chunks.append({'id':f"{uri}_{i}", 'uri':uri, 'text':c})
        
chunks

[{'id': 'https://docs.fastht.ml/explains/oauth.html_0',
  'uri': 'https://docs.fastht.ml/explains/oauth.html',
  'text': 'OAuth\nOAuth is an open standard for ‘access delegation’, commonly used as a way for Internet users to grant websites or applications access to their information on other websites but without giving them the passwords. It is the mechanism that enables “Log in with Google” on many sites, saving you from having to remember and manage yet another password. Like many auth-related topics, there’s a lot of depth and complexity to the OAuth standard, but once you understand the basic usage it can be a very convenient alternative to managing your own user accounts.\nOn this page you’ll see how to use OAuth with FastHTML to implement some common pieces of functionality.\nIn FastHTML you set up a client like GoogleAppClient\n. The client is responsible for storing the client ID and client secret, and for handling the OAuth flow. Let’s run through three examples, illustrating 

## Create a lancedb dump of the data

In [79]:
import pydantic,lancedb
from lancedb.pydantic import LanceModel, Vector
from pydantic import BaseModel
import numpy as np
from lancedb.embeddings import get_registry

"""connect to the db (file)"""
db = lancedb.connect('../docs_search/app/data/')


"""https://lancedb.github.io/lancedb/embeddings/#openai-embedding-function"""
func = get_registry().get("openai").create(name="text-embedding-ada-002")

class HelpContent(LanceModel):
    """a simple lance data model to wrap the embedding in IO"""
    id: str
    uri: str
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
        
"""Create a data model for content"""

table = db.create_table("fast_html_help",  schema=HelpContent, mode="overwrite")

table.add( chunks)

tbl = db.open_table('fast_html_help')
tbl.to_pandas()

,id,uri,text,vector
0,https://docs.fastht.ml/explains/oauth.html_0,https://docs.fastht.ml/explains/oauth.html,OAuth\nOAuth is an open standard for ‘access d...,"[0.009375877, 0.012944715, 0.003971441, -0.020..."
1,https://docs.fastht.ml/api/oauth.html_0,https://docs.fastht.ml/api/oauth.html,It can sometimes be useful to pass state to th...,"[-0.017146317, 0.009280515, 0.011246965, -0.02..."
2,https://docs.fastht.ml/explains/faq.html_0,https://docs.fastht.ml/explains/faq.html,FAQ\nFrequently Asked Questions\nWhy is FastHT...,"[0.0058852127, 0.020239817, 0.018659236, -0.02..."
3,https://docs.fastht.ml/ref/live_reload.html_0,https://docs.fastht.ml/ref/live_reload.html,Live Reloading\nWhen building your app it can ...,"[-0.027140377, 0.024017788, 0.015530403, -0.00..."
4,https://docs.fastht.ml/api/components.html_0,https://docs.fastht.ml/api/components.html,from lxml import html as lx\nfrom pprint impor...,"[0.0037392746, 0.03210846, -0.015109863, -0.00..."
5,https://docs.fastht.ml/explains/routes.html_0,https://docs.fastht.ml/explains/routes.html,from fasthtml.common import *\nRoutes\nBehavio...,"[0.0022127184, 0.016187962, -0.012949006, 0.00..."
6,https://docs.fastht.ml/tutorials/index.html_0,https://docs.fastht.ml/tutorials/index.html,Tutorials\nClick through to any of these tutor...,"[-0.012934649, 0.027691847, -0.0007663372, -0...."
7,https://docs.fastht.ml/api/js.html_0,https://docs.fastht.ml/api/js.html,'.body {color: green;}') light_media(\n<style>...,"[5.0814408e-05, 0.027402073, -0.0064114286, -0..."
8,https://docs.fastht.ml/explains/explaining_xt_...,https://docs.fastht.ml/explains/explaining_xt_...,from fasthtml.common import *\ndef example():\...,"[0.006444941, 0.03280558, -0.025793595, -0.031..."
9,https://docs.fastht.ml/api/core.html_0,https://docs.fastht.ml/api/core.html,import time\nfrom IPython import display\nfrom...,"[-0.005222517, 0.015373581, -0.011795122, 0.02..."


## Check the search basically works

In [81]:
"""check table directly"""
tbl.search('Is there a JS Walkthrough').select(['text']).to_list()

[{'text': 'Tutorials\nClick through to any of these tutorials to get started with FastHTML’s features.\nTitle | Description |\n---|---|\nFastHTML By Example | An introduction to FastHTML from the ground up, with four complete examples |\nWeb Devs Quickstart | A fast introduction to FastHTML for experienced web developers. |\nBYO Blog | Learn the foundations of FastHTML by creating your own blogging system from scratch. |\nJS App Walkthrough | How to build a website with custom JavaScript in FastHTML step-by-step |\nNo matching items',
  '_distance': 0.3811552822589874},
 {'text': 'JS App Walkthrough\nInstallation\nYou’ll need the following software to complete the tutorial, read on for specific installation instructions:\n- Python\n- A Python package manager such as pip (which normally comes with Python) or uv\n- FastHTML\n- Web browser\n- Railway.app account\nIf you haven’t worked with Python before, we recommend getting started with Miniconda.\nNote that you will only need to follow 

In [2]:
"""check config for the vector store wrapper in the server utils"""
from app.src.chat_utils import VectorStore
store = VectorStore()
store('Is there a JS Walkthrough')

{'status': 'found some data sorted by distance / quality - please use the data to answer the users question',
 'results': [{'text': 'Tutorials\nClick through to any of these tutorials to get started with FastHTML’s features.\nTitle | Description |\n---|---|\nFastHTML By Example | An introduction to FastHTML from the ground up, with four complete examples |\nWeb Devs Quickstart | A fast introduction to FastHTML for experienced web developers. |\nBYO Blog | Learn the foundations of FastHTML by creating your own blogging system from scratch. |\nJS App Walkthrough | How to build a website with custom JavaScript in FastHTML step-by-step |\nNo matching items',
   '_distance': 0.3811552822589874},
  {'text': 'JS App Walkthrough\nInstallation\nYou’ll need the following software to complete the tutorial, read on for specific installation instructions:\n- Python\n- A Python package manager such as pip (which normally comes with Python) or uv\n- FastHTML\n- Web browser\n- Railway.app account\nIf 

In [2]:
"""check the agent
1. calls the store
2. calls GPT (requires OPEN_AI key)
"""
from app.src.chat_utils import agent
from IPython.display import Markdown
Markdown(agent('what is Fast HTML',is_streaming=False))

FastHTML is a next-generation web framework designed for building fast and scalable web applications using minimal and compact code. It is built on Python, allowing developers to leverage the full power of the Python language and its ecosystem. FastHTML is characterized by its powerful and expressive capabilities, making it suitable for creating advanced and interactive web applications.

Key features of FastHTML include:

- **Simplicity**: FastHTML has a simple and intuitive syntax, making it easy to learn and use, which helps developers build complex applications quickly.
- **Performance**: The framework is designed to be fast and lightweight, enabling developers to write less code while achieving more functionality.
- **HTML and HTTP Mapping**: FastHTML's functionality directly maps to HTML and HTTP, allowing developers to encapsulate these technologies using good software engineering practices.

To get started with FastHTML, you can install it via pip with the command `pip install python-fasthtml`. The framework supports various features such as routing, session management, and custom components, making it a versatile choice for web development.

For more detailed information, you can refer to the official FastHTML documentation and tutorials available online.

In [3]:
Markdown(agent('Please give me an example snippet FastHTML python program',is_streaming=False))

Here's a simple example of a FastHTML Python program that creates a basic web application:

```python
from fasthtml.common import *

app, rt = fast_app()

@rt('/')
def get():
    return Div(H1('Hello, World!'), P('This is a simple FastHTML app.'))

serve()
```

### Explanation:
1. **Importing FastHTML**: The program starts by importing necessary components from the `fasthtml.common` module.
2. **Creating the App**: It initializes a FastHTML app using `fast_app()`.
3. **Defining a Route**: The `@rt('/')` decorator defines a route for the root URL (`/`). When this URL is accessed, the `get()` function is called.
4. **Returning HTML**: The `get()` function returns a `Div` containing an `H1` header and a `P` paragraph.
5. **Serving the App**: Finally, `serve()` is called to run the application.

To run this app, save the code in a file (e.g., `main.py`) and execute it with Python. You can then visit `http://localhost:5001` in your web browser to see the output.